In [1]:
# pip install requests
 
import requests
 
api_key = ""
filename = "주간농사정보 제27호(2025.7.14.~7.20.).pdf"
 
urls = "https://api.upstage.ai/v1/document-digitization"
headers1 = {"Authorization": f"Bearer {api_key}"}
files1 = {"document": open(filename, "rb")}
data1 = {"ocr": "force", "base64_encoding": "['table']", "model": "document-parse"}
response2 = requests.post(urls, headers=headers1, files=files1, data=data1)
 
print(response2.json())

{'api': '2.0', 'content': {'html': '<header id=\'0\' style=\'font-size:16px\'>농촌진흥청</header>\n<h1 id=\'1\' style=\'font-size:20px\'>제27호</h1>\n<br><h1 id=\'2\' style=\'font-size:22px\'>주간농사정보</h1>\n<h1 id=\'3\' style=\'font-size:18px\'>2025. 7. 14. ~ 7. 20.</h1>\n<footer id=\'4\' style=\'font-size:14px\'>농촌진흥청에서는 금주에 꼭 실천해야 할 주요 농업기술 정보를<br>농업인들에게 매주 신속하게 제공하고 있습니다</footer>\n<h1 id=\'5\' style=\'font-size:18px\'>목 차</h1>\n<p id=\'6\' data-category=\'paragraph\' style=\'font-size:14px\'>제1장 농업정보 1<br>제2장 벼 4<br>제3장 밭 작 물 6<br>제4장 채 소 8<br>제5장 과 수 . 11<br>제6장 화 훼 15<br>제7장 특용작물 18<br>제8장 축 산 20<br>제9장 양 봉 25</p>\n<h1 id=\'7\' style=\'font-size:18px\'>요 약</h1>\n<br><table id=\'8\' style=\'font-size:14px\'><thead><tr><td>분야</td><td>핵심기술 및 정보</td></tr></thead><tbody><tr><td>농업 정보</td><td>· (기상) 기온은 평년(23.8~25.2°C)보다 높겠고, 강수량은 평년(42.1~74.4mm) 보다 적겠음 * 주로 고기압의 영향을 받아 덥고 습하겠음 · (저수율) 63.6%(평년 63.8%의 99.7%) * 7. 7. 기준</td></tr><tr><td>벼</td><td>· (본답관리) 벼 이삭이 생기는 시기부터 팰 때까지 논물 걸러대기 실시, 이삭 패기 전 

In [2]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(response2.json().get("content")["html"], 'html.parser')

chunks = []
current_title = None
current_content = ""

for tag in soup.find_all(['h1', 'p', 'table']):
    text = tag.get_text(separator="\n").strip()
    
    # h1 중에서 큰 제목을 새 chunk의 기준으로 사용
    if tag.name == 'h1' and 'font-size' in tag.attrs.get('style', ''):
        font_size = int(tag['style'].split("font-size:")[1].replace("px", "").strip())

        if font_size >= 20:
            if current_title and current_content:
                chunks.append({"title": current_title, "content": current_content.strip()})
                current_content = ""

            current_title = text

        else:
            current_content += f"\n{text}"

    elif tag.name in ['p', 'table']:
        current_content += f"\n{text}"

# 마지막 chunk 저장
if current_title and current_content:
    chunks.append({"title": current_title, "content": current_content.strip()})


In [8]:
chunks

[{'title': '주간농사정보',
  'content': '2025. 7. 14. ~ 7. 20.\n목 차\n제1장 농업정보 1\n제2장 벼 4\n제3장 밭 작 물 6\n제4장 채 소 8\n제5장 과 수 . 11\n제6장 화 훼 15\n제7장 특용작물 18\n제8장 축 산 20\n제9장 양 봉 25\n요 약\n분야\n핵심기술 및 정보\n농업 정보\n· (기상) 기온은 평년(23.8~25.2°C)보다 높겠고, 강수량은 평년(42.1~74.4mm) 보다 적겠음 * 주로 고기압의 영향을 받아 덥고 습하겠음 · (저수율) 63.6%(평년 63.8%의 99.7%) * 7. 7. 기준\n벼\n· (본답관리) 벼 이삭이 생기는 시기부터 팰 때까지 논물 걸러대기 실시, 이삭 패기 전 전용복비 시용 · (병해충관리) 벼멸구, 잎도열병 우려로 조생종벼 출수 전 전용약제 초기 방제\n밭작물\n· (고온대비) 스프링클러 가동, 토양 피복으로 지온 상승 억제, 고온성 해충 발생 증가에 대비해 예찰 및 방제 실시 · (콩) 생육상황 고려 웃거름 시비, 병해충 방제 철저, 나방류 적기 방제 · (옥수수) 적기 수확, 풋옥수수 소비자 공급까지 저온 유지 · (참깨·들깨) 역병, 시들음병, 잎마름병, 잎말이나방 위주 방제\n채소\n· (고추) 고온 대비 적정 토양 수분 유지, 탄저병 방제, 엽면시비, 적기수확 등 · (여름 배추·무) 무름병, 석회결핍, 진딧물 방제 등 폭염 대비 관리 · (양파) 7월 말에서 8월 고온기에 40일 이상 육묘상 태양열 소독 · (시설채소) 차광망 및 환기팬 활용 고온 장해 예방, 병해충 방제 철저\n과수\n· (햇볕데임) 일 최고기온 31°C 이상 맑은 날 발생, 양광면 음광면에 비해 10°C 높음 - 적절한 가지 배치를 통한 일소피해 예방, 미세살수장치 가동 필요 · (과원관리) 관·배수 철저로 열과 방지, 배수로 정비, 경사지 과원 토양유실 방지 · (하계전정) 웃자람가지 제거로 일조량 확보, 7월 중순경 전정 통한 꽃눈형성 · (병해충 방제) 겹무늬

In [9]:
import boto3

# MinIO 설정
minio_endpoint = "http://artifact-store:9000"  # 도커 내부라면 'host.docker.internal:9000'
access_key = "minio"
secret_key = "miniostorage"

# S3 클라이언트 생성 (endpoint_url 반드시 지정)
s3_client = boto3.client(
    's3',
    endpoint_url=minio_endpoint,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
)


In [10]:
import time
import uuid

today = time.strftime('%Y-%m-%d')
doc_id = time.strftime('%Y-%m-%d_')+str(uuid.uuid4())[:10]

In [12]:
for i, chunk in enumerate(chunks):
    chunk["chunk_id"] = f"{doc_id}_{i:03}"
    chunk["date"] = today

In [15]:
import io 
import json 

buffer = io.BytesIO()
buffer.write(json.dumps(chunks, ensure_ascii=False).encode("utf-8"))
buffer.seek(0)

# 저장 파일명
filename = f"data/{doc_id}.json"

s3_client.upload_fileobj(buffer, "chunk", filename)
print("업로드 완료")

업로드 완료


In [16]:
raw_buffer = io.BytesIO() 


raw_buffer.write(json.dumps(response2.json(), ensure_ascii=False).encode("utf-8"))
raw_buffer.seek(0)

raw_id= time.strftime('%Y-%m-%d_')+str(uuid.uuid4())[:10]

raw_files = f"raw_data/{raw_id}.json"
raw_files

s3_client.upload_fileobj(raw_buffer, "ocr-raw", raw_files)